In [1]:
from pynq import PL
from pynq import Overlay

In [2]:
PL.reset()
overlay = Overlay('pynq_int.bit')

In [3]:
timer1 = overlay.timer_1
timer1._interrupts

{'interrupt': {'controller': 'axi_intc_0',
  'index': 1,
  'fullpath': 'timer_1/interrupt'}}

In [4]:
timer0 = overlay.timer_0
timer0._interrupts

{'interrupt': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'timer_0/interrupt'}}

In [5]:
gpio0 = overlay.gpio_0
gpio0._interrupts

{'ip2intc_irpt': {'controller': 'axi_intc_0',
  'index': 2,
  'fullpath': 'gpio_0/ip2intc_irpt'}}

In [6]:
int_tim0 = timer0.interrupt
int_tim1 = timer1.interrupt

In [7]:
timer0.register_map

RegisterMap {
  TCSR0 = Register(MDT0=0, UDT0=0, GENT0=0, CAPT0=0, ARHT0=0, LOAD0=0, ENIT0=0, ENT0=0, T0INT=0, PWMA0=0, ENALL=0, CASC=0),
  TLR0 = Register(TCLR0=0),
  TCR0 = Register(TCR0=0),
  TCSR1 = Register(MDT1=0, UDT1=0, GENT1=0, CAPT1=0, ARHT1=0, LOAD1=0, ENIT1=0, ENT1=0, T1INT=0, PWMA1=0, ENALL=0),
  TLR1 = Register(TCLR1=0),
  TCR1 = Register(TCR1=0)
}

In [8]:
gpio0.register_map.IP_ISR.Channel_2_Interrupt_Status = 1
gpio0.register_map.IP_IER.Channel_1_Interrupt_Enable = 1
gpio0.register_map.GIER.Global_Interrupt_Enable = 1
gpio0.register_map.GPIO2_TRI.Channel_2_GPIO_TRI = 1

gpio0.register_map


RegisterMap {
  GPIO_DATA = Register(Channel_1_GPIO_DATA=0),
  GPIO_TRI = Register(Channel_1_GPIO_TRI=1),
  GPIO2_DATA = Register(Channel_2_GPIO_DATA=0),
  GPIO2_TRI = Register(Channel_2_GPIO_TRI=4294967295),
  GIER = Register(Global_Interrupt_Enable=1),
  IP_ISR = Register(Channel_1_Interrupt_Status=0, Channel_2_Interrupt_Status=0),
  IP_IER = Register(Channel_1_Interrupt_Enable=1, Channel_2_Interrupt_Enable=0)
}

In [9]:
gpio0.channel1.setdirection('in')  # Establecer el canal 1 como entrada
print(gpio0.has_interrupts)  # Verificar si hay interrupciones configuradas

True


In [10]:
# Leer el estado de la interrupción para Channel 1
print(gpio0.register_map.IP_ISR.Channel_1_Interrupt_Status)

# Escribir 1 para habilitar la interrupción de Channel 1
gpio0.register_map.IP_IER.Channel_1_Interrupt_Enable = 1

# Escribir 0 para limpiar la interrupción de Channel 1
gpio0.register_map.IP_ISR.Channel_1_Interrupt_Status = 0

0


In [11]:
print(dir(gpio0))

['Channel', 'InOut', 'Input', 'Output', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call', '_channels', '_fullpath', '_gpio', '_interrupts', '_register_map', '_register_name', '_registers', '_setup_packet_prototype', '_start_ert', '_start_sw', 'bindto', 'channel1', 'channel2', 'device', 'has_interrupts', 'ip2intc_irpt', 'mmio', 'read', 'register_map', 'setdirection', 'setlength', 'signature', 'write']


In [12]:
import asyncio

async def wait_for_gpio0():
    # Limpiar el estado de la interrupción
    gpio0.register_map.IP_ISR.Channel_1_Interrupt_Status = 0
    gpio0.register_map.IP_IER.Channel_1_Interrupt_Enable = 1
    gpio0.register_map.GIER.Global_Interrupt_Enable = 1

    # Bucle para esperar a que la interrupción ocurra
    while True:
        # Comprobar si la interrupción ha ocurrido (Estado de la interrupción en 1)
        if gpio0.register_map.IP_ISR.Channel_1_Interrupt_Status == 1:
            print("¡Interrupción ocurrida en Channel 1!")
            
            # Limpiar el estado de la interrupción (esto debería resetear el bit de la interrupción)
            gpio0.register_map.IP_ISR.Channel_1_Interrupt_Status = 1

            # Deshabilitar la interrupción temporalmente para evitar re-activaciones inesperadas
            gpio0.register_map.IP_IER.Channel_1_Interrupt_Enable = 0

            # Habilitar la interrupción nuevamente después de limpiarla
            gpio0.register_map.IP_IER.Channel_1_Interrupt_Enable = 1

            # Asegurar que la interrupción global sigue habilitada
            gpio0.register_map.GIER.Global_Interrupt_Enable = 1
            
            break
        
        # Esperar un poco antes de verificar nuevamente
        await asyncio.sleep(0.1)

# Crear y ejecutar el evento asincrónico
loop = asyncio.get_event_loop()
task = loop.create_task(wait_for_gpio0())
loop.run_until_complete(task)
    

¡Interrupción ocurrida en Channel 1!


In [13]:
print(dir(timer0))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call', '_fullpath', '_gpio', '_interrupts', '_register_map', '_register_name', '_registers', '_setup_packet_prototype', '_start_ert', '_start_sw', 'device', 'interrupt', 'mmio', 'read', 'register_map', 'signature', 'write']


In [14]:
async def wait_for_timer1(cycles):
    timer1.register_map.TLR0 = cycles
    timer1.register_map.TCSR0.LOAD0 = 1
    timer1.register_map.TCSR0.LOAD0 = 0
    timer1.register_map.TCSR0.ENIT0 = 1
    timer1.register_map.TCSR0.ENT0 = 1
    timer1.register_map.TCSR0.UDT0 = 1
    await timer1.interrupt.wait()
    timer1.register_map.TCSR0.T0INT = 1

In [15]:
import asyncio
loop = asyncio.get_event_loop()

In [16]:
task = loop.create_task(wait_for_timer1(500000000))
loop.run_until_complete(task)